In [ ]:
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
import json
import MetaTrader5 as mt5

# Initialize MetaTrader 5
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()

def place_trade(symbol, volume, sl, tp, trade_type):
    # Define the trade request
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "sl": sl,
        "tp": tp,
        "deviation": 10,
        "magic": 123456,
        "type": mt5.ORDER_TYPE_BUY if trade_type == 'buy' else mt5.ORDER_TYPE_SELL,
        "type_filling": mt5.ORDER_FILLING_FOK
    }

    # Send the trade request
    result = mt5.order_send(request)

    return result

def check_profit_or_loss(initial_balance):
    # Get the new balance from the MT5 terminal
    account_info = mt5.account_info()
    new_balance = account_info.balance

    if initial_balance < new_balance:
        return "profit"
    elif initial_balance > new_balance:
        return "loss"
    else:
        return "no change"

def check_open_positions():
    # Get the open positions from MT5
    positions = mt5.positions_get()
    return len(positions) > 0

@csrf_exempt
def place_buy_trade(request):
    if request.method == 'POST':
        if check_open_positions():
            return JsonResponse({'status': 'error', 'message': 'There is already an open trade'}, status=400)

        data = json.loads(request.body)
        symbol = data['symbol']
        volume = data['volume']
        sl = data['sl']
        tp = data['tp']

        # Get the initial balance
        account_info = mt5.account_info()
        initial_balance = account_info.balance

        result = place_trade(symbol, volume, sl, tp, 'buy')

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            return JsonResponse({'status': 'error', 'retcode': result.retcode, 'comment': result.comment}, status=500)
        else:
            trade_status = check_profit_or_loss(initial_balance)
            return JsonResponse({'status': 'success', 'order': result.order, 'retcode': result.retcode, 'trade_status': trade_status}, status=200)

@csrf_exempt
def place_sell_trade(request):
    if request.method == 'POST':
        if check_open_positions():
            return JsonResponse({'status': 'error', 'message': 'There is already an open trade'}, status=400)

        data = json.loads(request.body)
        symbol = data['symbol']
        volume = data['volume']
        sl = data['sl']
        tp = data['tp']

        # Get the initial balance
        account_info = mt5.account_info()
        initial_balance = account_info.balance

        result = place_trade(symbol, volume, sl, tp, 'sell')

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            return JsonResponse({'status': 'error', 'retcode': result.retcode, 'comment': result.comment}, status=500)
        else:
            trade_status = check_profit_or_loss(initial_balance)
            return JsonResponse({'status': 'success', 'order': result.order, 'retcode': result.retcode, 'trade_status': trade_status}, status=200)
